In [0]:
landing_zone = "dbfs:/FileStore/retail_data"
orders_data = landing_zone + "/orders_data"
checkpoint_path = landing_zone + "/orders_checkpoint"

In [0]:
dbutils.fs.rm("dbfs:/FileStore/retail_data/orders_checkpoint",True)

In [0]:
from pyspark.sql.functions import input_file_name,current_timestamp

In [0]:
orders_df = spark.readStream \
.format("cloudFiles") \
.option("cloudFiles.format","csv") \
.option("cloudFiles.inferColumnTypes","true") \
.option("cloudFiles.schemaLocation", checkpoint_path) \
.option("header",True) \
.load(orders_data) \
.withColumn("file_name",input_file_name()) \
.withColumn("time_of_load",current_timestamp())

In [0]:
orders_df.display()

In [0]:
orders_streaming_query = orders_df.writeStream \
.format("delta") \
.option("checkpointLocation",checkpoint_path) \
.queryName("orders_streaming_query") \
.outputMode("append") \
.toTable("orderdeltanewOne")

In [0]:
orders_streaming_query.stop()

In [0]:
%sql
describe orderdeltanewOne